<a href="http://landlab.github.io"><img style="float: left" src="../../landlab_header.png"></a>

# Profiling and Scaling Analysis of the NetworkSedimentTransporter

Part 1: Do some stuff to give us generic, variable sized grids and parcels. 
Part 2: Do some profiling. 
Part 2: Do some scaling analysis.

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import numpy as np
from landlab.components import FlowDirectorSteepest, NetworkSedimentTransporter
from landlab.data_record import DataRecord
from landlab.grid.network import NetworkModelGrid
from landlab.plot import graph

import cProfile
import pstats
from pstats import SortKey
import io

import xarray as xr

import pandas as pd
import time

## Part 1: Variably-sized grids and parcels

### Part 1a: Grids

First, we need the ability to create different sizes of grids. 

A simple approach is to create a generic grid in which each node has two recievers. Lets start by writing a function that creates the x and y node coordinates and the linking structure for a given number of layers.   

I haven't tried to optimize this at all. 

In [ ]:
def create_node_xy_and_links(n_layers, x0=0., y0=0., xperc=0.9, dy=1.):
    assert xperc<1.0
    nodes_per_layer = np.power(2, np.arange(n_layers+1))
    nnodes = np.sum(nodes_per_layer)   
    x_of_node=[x0]
    y_of_node=[y0]
    nodes_at_link = []
    id_start_layer = 0
    for nl in np.arange(1, n_layers+1):
        nodes_last_layer = np.power(2, nl-1)
        nodes_this_layer = np.power(2, nl)              
        dx = xperc * (dy)*(0.5**(nl-1))
        for ni in range(nodes_last_layer):
            head_id = id_start_layer+ni            
            tail_id = len(x_of_node)           
            x = x_of_node[head_id]
            y = y_of_node[head_id]            
            x_of_node.extend([x-dx, x+dx])
            y_of_node.extend([y+dy, y+dy])            
            nodes_at_link.extend([(head_id, tail_id), (head_id, tail_id +1)])   
        id_start_layer = len(x_of_node) - nodes_this_layer
    return x_of_node, y_of_node, nodes_at_link

Lets demonstrate the different sorts of grids we get with different numbers of layers

In [ ]:
example_layers = [1, 3, 5, 7, 9]

nodes = []
for i, n_layers in enumerate(example_layers):
    x_of_node, y_of_node, nodes_at_link = create_node_xy_and_links(n_layers)
    grid = NetworkModelGrid((y_of_node, x_of_node), nodes_at_link)

    graph.plot_graph(grid, at="node,link", with_id=False)
    nodes.append(grid.number_of_nodes)

In [ ]:
plt.plot(example_layers, nodes)

### 1b: Generic grid.

The grid needs some additional fields added to it...

In [ ]:
def create_nmg_and_fd(n_layers):
    x_of_node, y_of_node, nodes_at_link = create_node_xy_and_links(n_layers)
    grid = NetworkModelGrid((y_of_node, x_of_node), nodes_at_link)

    _ = grid.add_field("topographic__elevation", grid.y_of_node.copy(), at="node")
    _ = grid.add_field("bedrock__elevation", grid.y_of_node.copy(), at="node")

    _ = grid.add_field("reach_length", 200.*np.ones(grid.number_of_links), at="link")  # m
    _ = grid.add_field("channel_width", 1.*np.ones(grid.number_of_links), at="link")  # m
    
    fd = FlowDirectorSteepest(grid)
    fd.run_one_step()
    return grid, fd

### 1c: Generic sets of parcels
A real application would likley use 50k+ parcels (b/c you want the full gsd) and 100-500 links in the network. 
You typically want at least 100 parcels per link. 

In [ ]:
def create_parcels(grid, parcels_per_link=5):

    # element_id is the link on which the parcel begins. 
    element_id = np.repeat(np.arange(grid.number_of_links), parcels_per_link)
    element_id = np.expand_dims(element_id, axis=1)

    volume = 1*np.ones(np.shape(element_id))  # (m3)
    active_layer = np.ones(np.shape(element_id)) # 1= active, 0 = inactive
    density = 2650 * np.ones(np.size(element_id))  # (kg/m3)
    abrasion_rate = 0 * np.ones(np.size(element_id)) # (mass loss /m)

    # Lognormal GSD
    medianD = 0.085 # m
    mu = np.log(medianD)
    sigma = np.log(2) #assume that D84 = sigma*D50
    np.random.seed(0)
    D = np.random.lognormal(
        mu,
        sigma,
        np.shape(element_id)
    )  # (m) the diameter of grains in each parcel

    time_arrival_in_link = np.random.rand(np.size(element_id), 1) 
    location_in_link = np.random.rand(np.size(element_id), 1) 

    variables = {
        "abrasion_rate": (["item_id"], abrasion_rate),
        "density": (["item_id"], density),
        "time_arrival_in_link": (["item_id", "time"], time_arrival_in_link),
        "active_layer": (["item_id", "time"], active_layer),
        "location_in_link": (["item_id", "time"], location_in_link),
        "D": (["item_id", "time"], D),
        "volume": (["item_id", "time"], volume)
    }


    items = {"grid_element": "link", "element_id": element_id}

    _OUT_OF_NETWORK = NetworkModelGrid.BAD_INDEX - 1

    parcels = DataRecord(
        grid,
        items=items,
        time=[0.0],
        data_vars=variables,
        dummy_elements={"link": [_OUT_OF_NETWORK]},
    )

    return parcels

In [ ]:
def create_all(nlayer=3, parcels_per_link=100, timesteps=10):

    grid, fd = create_nmg_and_fd(nlayer)

    parcels = create_parcels(grid, parcels_per_link=parcels_per_link)

    dt = 60 * 60 * 24 *12 # length of timestep (seconds) 
    flow_depth =  2.5 * np.ones([timesteps + 1, grid.number_of_links])
    
    return grid, fd, parcels, flow_depth, dt, timesteps

In [ ]:
grid, fd, parcels, flow_depth, dt, timesteps = create_all(nlayer=4, parcels_per_link=50, timesteps=10)

nst = NetworkSedimentTransporter(    
    grid,
    parcels,
    fd,
    flow_depth,
    bed_porosity=0.3,
    g=9.81,
    fluid_density=1000,
    transport_method="WilcockCrowe",
)

pr = cProfile.Profile()
pr.enable()
for t in range(0, (timesteps * dt), dt):
    nst.run_one_step(dt)
pr.disable()
s = io.StringIO()
sortby = SortKey.CUMULATIVE
ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
ps.print_stats()
#print(s.getvalue())

with open("profile.txt", "w") as f:
    f.write(s.getvalue())

In [ ]:
break

In [ ]:
def profile_code(nlayer=3, parcels_per_link=100, timesteps=10):

    grid, fd = create_nmg_and_fd(nlayer)

    parcels = create_parcels(grid, parcels_per_link=parcels_per_link)

    dt = 60 * 60 * 24 *12 # length of timestep (seconds) 
    flow_depth =  2.5 * np.ones([timesteps + 1, grid.number_of_links])

    init_start = time.time()
    nst = NetworkSedimentTransporter(    
        grid,
        parcels,
        fd,
        flow_depth,
        bed_porosity=0.3,
        g=9.81,
        fluid_density=1000,
        transport_method="WilcockCrowe",
    )
    init_duration = time.time() - init_start

    partition = []
    move_parcel = []
    
    for t in range(timesteps):
        nst._time += dt

        nst._time_idx += 1
        nst._create_new_parcel_time()

        if nst._this_timesteps_parcels.any():
            
            partition_start = time.time()
            nst._partition_active_and_storage_layers()
            partition.append(time.time()-partition_start)
            
            nst._adjust_node_elevation()
            nst._update_channel_slopes()
            nst._update_transport_time()
            
            move_start = time.time()
            nst._move_parcel_downstream(dt)
            move_parcel.append(time.time()-move_start)
            
        else:
            msg = "No more parcels on grid"
            raise RuntimeError(msg)

    return (grid.number_of_nodes, parcels_per_link, timesteps), init_duration, partition, move_parcel


In [ ]:
(nn, ppl, ts), init, part, move = profile_code(
    nlayer=3, 
    parcels_per_link=100, 
    timesteps=10)

In [ ]:
out = {}
for i in range(2, 7):
    for j in [10, 20, 50, 100, 200, 500, 1000, 2000]:
        (nn, ppl, ts), init, part, move = profile_code(
            nlayer=i, 
            parcels_per_link=j, 
            timesteps=0)
        out[(nn, ppl)] = {"init": init}

In [ ]:
df = pd.DataFrame.from_dict(out)
df.head()

In [ ]:
out

In [ ]:
nst._time += dt
nst._time_idx += 1
nst._create_new_parcel_time()

nst._partition_active_and_storage_layers()
nst._adjust_node_elevation()
nst._update_channel_slopes()
nst._update_transport_time()


pr = cProfile.Profile()
pr.enable()
nst._move_parcel_downstream(dt)
pr.disable()
s = io.StringIO()
sortby = SortKey.CUMULATIVE
ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
ps.print_stats()
#print(s.getvalue())


In [ ]:
with open("profile_move_downstream.txt", "w") as f:
    f.write(s.getvalue())

In [ ]:
grid.number_of_links

In [ ]:
parcels._dataset